In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from matplotlib.widgets import Slider
import tensorflow as tf
import keras
from keras.layers import Dense
from keras.models import Sequential
from sklearn.datasets import make_blobs
import logging

logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [ ]:
SEED = 30

keras.utils.set_random_seed(SEED)
np.random.seed(SEED)

In [ ]:
def my_softmax(z):
    z = np.asarray(z, dtype=np.float32)
    z_shifted = z - np.max(z)
    exp_z = np.exp(z_shifted)
    return exp_z / np.sum(exp_z)

In [ ]:
def plt_softmax(fn):
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    plt.subplots_adjust(bottom=0.35)

    axz0 = fig.add_axes([0.15, 0.10, 0.30, 0.03])  # [left, bottom, width, height]
    axz1 = fig.add_axes([0.15, 0.15, 0.30, 0.03])
    axz2 = fig.add_axes([0.15, 0.20, 0.30, 0.03])
    axz3 = fig.add_axes([0.15, 0.25, 0.30, 0.03])

    z = np.array(["z0", "z1", "z2", "z3"])
    z0 = Slider(axz0, z[0], 0, 10, valinit=1, valstep=0.1)
    z1 = Slider(axz1, z[1], 0, 10, valinit=2, valstep=0.1)
    z2 = Slider(axz2, z[2], 0, 10, valinit=3, valstep=0.1)
    z3 = Slider(axz3, z[3], 0, 10, valinit=4, valstep=0.1)

    z_vals = [z0.val, z1.val, z2.val, z3.val]
    zbars_c = ax[0].barh(z, height=0.6, width=z_vals, left=None, align="center")
    zbars = zbars_c.get_children()
    ax[0].set_xlim(0, 10)
    ax[0].set_title("z input to softmax")

    anames = np.array(["a0", "a1", "a2", "a3"])
    a_vals = fn(np.array(z_vals))
    abars_c = ax[1].barh(
        anames, height=0.6, width=a_vals, left=None, align="center", color="darkred"
    )
    abars = abars_c.get_children()
    ax[1].set_xlim(0, 1)
    ax[1].set_title("softmax(z)")

    def update(val):
        z_vals = [z0.val, z1.val, z2.val, z3.val]
        for i in range(len(z_vals)):
            zbars[i].set_width(z_vals[i])
        a_vals = fn(np.array(z_vals))
        for i in range(len(a_vals)):
            abars[i].set_width(a_vals[i])
        fig.canvas.draw_idle()

    z0.on_changed(update)
    z1.on_changed(update)
    z2.on_changed(update)
    z3.on_changed(update)

    plt.show()

In [ ]:
plt.close("all")
plt_softmax(my_softmax)

In [ ]:
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
X_train, y_train = make_blobs(
    n_samples=2000, centers=centers, cluster_std=1.0, random_state=SEED
)

In [ ]:
model = Sequential(
    [
        Dense(25, activation="relu"),
        Dense(15, activation="relu"),
        Dense(4, activation="softmax"),
    ]
)

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
)

In [ ]:
model.fit(X_train, y_train, epochs=10)

In [ ]:
p_nonpreferred = model.predict(X_train)
print(p_nonpreferred[:2])
print(
    f"largest probability : {np.max(p_nonpreferred)}\nsmallest probability : {np.min(p_nonpreferred)}"
)

In [ ]:
preferred_model = Sequential(
    [
        Dense(25, activation="relu"),
        Dense(15, activation="relu"),
        Dense(4, activation="linear"),
    ]
)

In [ ]:
preferred_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(0.001),
)

In [ ]:
preferred_model.fit(X_train, y_train, epochs=10)

In [ ]:
z_preferred = preferred_model.predict(X_train)
print(z_preferred[:2])
print(f"largest logit : {np.max(z_preferred)}\nsmallest logit : {np.min(z_preferred)}")

In [ ]:
p_preferred = keras.activations.softmax(z_preferred)
print(f"two example output vectors:\n {p_preferred[:2]}")
print(
    f"largest probability : {np.max(p_preferred)}\nsmallest probability : {np.min(p_preferred)}"
)

In [ ]:
print(np.max(np.abs(p_preferred)))
print(np.max(np.abs(p_nonpreferred)))

In [ ]:
for i in range(5):
    print(f"{p_preferred[i]}, category {np.argmax(p_preferred[i])}")

In [ ]:
print(X_train[1], y_train[1], p_preferred[1].numpy(), np.argmax(p_preferred[1]))